In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt

import json
import git
from collections import defaultdict
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
cd ../cf-graph-countyfair/

/home/christopher/dev/conda-forge/cf-graph-countyfair


In [4]:
license_lengths = {}

In [5]:
repo = git.Repo('.')
ggit = repo.git

In [9]:
for commit in tqdm(repo.iter_commits(rev='afaec18c2667f94cd06c6ed3e4c39a076e8acf14...master')):
    timestamp = commit.committed_date
    ggit.checkout(commit)
    a = !wc -l status/unlicensed.json
    license_lengths[timestamp] = int(a[0].split(' ')[0])

0it [00:00, ?it/s]


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-list afaec18c2667f94cd06c6ed3e4c39a076e8acf14...HEAD --
  stderr: 'fatal: Invalid symmetric difference expression afaec18c2667f94cd06c6ed3e4c39a076e8acf14...HEAD
'

In [ ]:
cd ../by-the-numbers

In [ ]:
len(license_lengths)

In [ ]:
df = pd.DataFrame.from_dict([{'time': k, 'count': v} for k, v in license_lengths.items()])

In [ ]:
df.head()

In [ ]:
df.index = pd.to_datetime(df['time'], unit='s')

In [ ]:
df.index = pd.to_datetime(df.index, utc=True)

In [ ]:
df.index = pd.DatetimeIndex(df.index)

In [ ]:
df.head()

In [ ]:
df3 = df.resample('D').mean()

In [ ]:
df3.head()

In [ ]:
for context in [("dark_background", "seaborn-talk"), ("seaborn-talk", )]:
    with plt.style.context(*context):
        fig, ax = plt.subplots()
        df3.plot(y='count', ax=ax)
        ax.set_title(f"Number of Unlicensed packages")
        ax.set_xlabel("Date (Aggregated by Day)")
        ax.set_ylabel("Number of Packages")
        fig.savefig(f"plots/license-{'_'.join(context)}.png", bbox_inches='tight', transparent=True)
    